In [1]:
!pip install openml

    100% |################################| 163kB 4.6MB/s 
  Running setup.py bdist_wheel for openml ... done
  Stored in directory: /home/idies/.cache/pip/wheels/71/ec/5f/aaad9e184680b0b8f1a02ff0ec640cace5adf5bff7bb0af1b4
  Running setup.py bdist_wheel for liac-arff ... done
  Stored in directory: /home/idies/.cache/pip/wheels/d1/6a/e7/529dc54d76ecede4346164a09ae3168df358945612710f5203
Successfully built openml liac-arff


In [2]:
!git clone https://gitlab.com/mirsakhawathossain/pha-ml.git

Cloning into 'pha-ml'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 17 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (17/17), done.


In [34]:
import openml
from sklearn import neighbors
import pandas as pd

openml.config.start_using_configuration_for_example()

In [35]:
openml.config.apikey = 'ba071d55fd6b48f947739f6348b55039'
openml.config.server ='https://www.openml.org/api/v1'

In [59]:
dataset = openml.datasets.get_dataset(42252)
print(dataset)

OpenML Dataset
Name..........: Asteroid_Dataset
Version.......: 1
Format........: arff
Upload Date...: 2020-02-16 12:21:29
Licence.......: Public
Download URL..: https://www.openml.org/data/v1/download/21799723/Asteroid_Dataset.arff
OpenML URL....: https://www.openml.org/d/42252
# of features.: 35
# of instances: 126131


In [37]:
tasks = openml.tasks.list_tasks(task_type_id=1)

In [38]:
tasks = pd.DataFrame.from_dict(tasks, orient='index')
print(tasks.columns)
print(f"First 5 of {len(tasks)} tasks:")
print(tasks.head())

# As conversion to a pandas dataframe is a common task, we have added this functionality to the
# OpenML-Python library which can be used by passing ``output_format='dataframe'``:
tasks_df = openml.tasks.list_tasks(task_type_id=1, output_format='dataframe')
print(tasks_df.head())

Index(['tid', 'ttid', 'did', 'name', 'task_type', 'status',
       'estimation_procedure', 'evaluation_measures', 'source_data',
       'target_feature', 'MajorityClassSize', 'MaxNominalAttDistinctValues',
       'MinorityClassSize', 'NumberOfClasses', 'NumberOfFeatures',
       'NumberOfInstances', 'NumberOfInstancesWithMissingValues',
       'NumberOfMissingValues', 'NumberOfNumericFeatures',
       'NumberOfSymbolicFeatures', 'cost_matrix'],
      dtype='object')
First 5 of 3308 tasks:
   tid  ttid  did        name                  task_type  status  \
2    2     1    2      anneal  Supervised Classification  active   
3    3     1    3    kr-vs-kp  Supervised Classification  active   
4    4     1    4       labor  Supervised Classification  active   
5    5     1    5  arrhythmia  Supervised Classification  active   
6    6     1    6      letter  Supervised Classification  active   

      estimation_procedure  evaluation_measures source_data target_feature  \
2  10-fold Crossval

In [39]:
tasks = openml.tasks.list_tasks(data_id=42252, output_format='dataframe')
print(f"First 5 of {len(tasks)} tasks:")
print(tasks.head())

First 5 of 1 tasks:
           tid  ttid    did              name                  task_type  \
211687  211687     1  42252  Asteroid_Dataset  Supervised Classification   

        status     estimation_procedure  evaluation_measures source_data  \
211687  active  10-fold Crossvalidation  predictive_accuracy       42252   

       target_feature  MajorityClassSize  MinorityClassSize  NumberOfClasses  \
211687            pha             125975                156                2   

        NumberOfFeatures  NumberOfInstances  \
211687                34             126131   

        NumberOfInstancesWithMissingValues  NumberOfMissingValues  \
211687                                  96                     99   

        NumberOfNumericFeatures  NumberOfSymbolicFeatures  
211687                       31                         2  


In [40]:
task_id = 211687
task = openml.tasks.get_task(task_id)


In [41]:
n_repeats, n_folds, n_samples = task.get_split_dimensions()

In [42]:
print(
    'Task {}: number of repeats: {}, number of folds: {}, number of samples {}.'.format(
        task_id, n_repeats, n_folds, n_samples,
    )
)

Task 211687: number of repeats: 1, number of folds: 10, number of samples 1.


In [24]:
train_indices, test_indices = task.get_train_test_split_indices(
    repeat=0,
    fold=0,
    sample=0,
)

print(train_indices.shape, train_indices.dtype)
print(test_indices.shape, test_indices.dtype)

(113517,) int32
(12614,) int32


In [43]:
X, y, _, _ = task.get_dataset().get_data(task.target_name)
X_train = X.loc[train_indices]
y_train = y[train_indices]
X_test = X.loc[test_indices]
y_test = y[test_indices]

print(
    'X_train.shape: {}, y_train.shape: {}, X_test.shape: {}, y_test.shape: {}'.format(
        X_train.shape, y_train.shape, X_test.shape, y_test.shape,
    )
)

X_train.shape: (113518, 33), y_train.shape: (113518,), X_test.shape: (12613, 33), y_test.shape: (12613,)


In [44]:
task_id = 211687
task = openml.tasks.get_task(task_id)
n_repeats, n_folds, n_samples = task.get_split_dimensions()
print(
    'Task {}: number of repeats: {}, number of folds: {}, number of samples {}.'.format(
        task_id, n_repeats, n_folds, n_samples,
    )
)


Task 211687: number of repeats: 1, number of folds: 10, number of samples 1.


In [45]:
for repeat_idx in range(n_repeats):
    for fold_idx in range(n_folds):
        for sample_idx in range(n_samples):
            train_indices, test_indices = task.get_train_test_split_indices(
                repeat=repeat_idx,
                fold=fold_idx,
                sample=sample_idx,
            )
            X_train = X.loc[train_indices]
            y_train = y[train_indices]
            X_test = X.loc[test_indices]
            y_test = y[test_indices]

            print(
                'Repeat #{}, fold #{}, samples {}: X_train.shape: {}, '
                'y_train.shape {}, X_test.shape {}, y_test.shape {}'.format(
                    repeat_idx, fold_idx, sample_idx, X_train.shape, y_train.shape, X_test.shape,
                    y_test.shape,
                )
            )

Repeat #0, fold #0, samples 0: X_train.shape: (113517, 33), y_train.shape (113517,), X_test.shape (12614, 33), y_test.shape (12614,)
Repeat #0, fold #1, samples 0: X_train.shape: (113518, 33), y_train.shape (113518,), X_test.shape (12613, 33), y_test.shape (12613,)
Repeat #0, fold #2, samples 0: X_train.shape: (113518, 33), y_train.shape (113518,), X_test.shape (12613, 33), y_test.shape (12613,)
Repeat #0, fold #3, samples 0: X_train.shape: (113518, 33), y_train.shape (113518,), X_test.shape (12613, 33), y_test.shape (12613,)
Repeat #0, fold #4, samples 0: X_train.shape: (113518, 33), y_train.shape (113518,), X_test.shape (12613, 33), y_test.shape (12613,)
Repeat #0, fold #5, samples 0: X_train.shape: (113518, 33), y_train.shape (113518,), X_test.shape (12613, 33), y_test.shape (12613,)
Repeat #0, fold #6, samples 0: X_train.shape: (113518, 33), y_train.shape (113518,), X_test.shape (12613, 33), y_test.shape (12613,)
Repeat #0, fold #7, samples 0: X_train.shape: (113518, 33), y_train.s

In [48]:
task_id = 211687
task = openml.tasks.get_task(task_id)
n_repeats, n_folds, n_samples = task.get_split_dimensions()
print(
    'Task {}: number of repeats: {}, number of folds: {}, number of samples {}.'.format(
        task_id, n_repeats, n_folds, n_samples,
    )
)

Task 211687: number of repeats: 1, number of folds: 10, number of samples 1.


In [49]:
for repeat_idx in range(n_repeats):
    for fold_idx in range(n_folds):
        for sample_idx in range(n_samples):
            train_indices, test_indices = task.get_train_test_split_indices(
                repeat=repeat_idx,
                fold=fold_idx,
                sample=sample_idx,
            )
            X_train = X.loc[train_indices]
            y_train = y[train_indices]
            X_test = X.loc[test_indices]
            y_test = y[test_indices]

            print(
                'Repeat #{}, fold #{}, samples {}: X_train.shape: {}, '
                'y_train.shape {}, X_test.shape {}, y_test.shape {}'.format(
                    repeat_idx, fold_idx, sample_idx, X_train.shape, y_train.shape, X_test.shape,
                    y_test.shape,
                )
            )

Repeat #0, fold #0, samples 0: X_train.shape: (113517, 33), y_train.shape (113517,), X_test.shape (12614, 33), y_test.shape (12614,)
Repeat #0, fold #1, samples 0: X_train.shape: (113518, 33), y_train.shape (113518,), X_test.shape (12613, 33), y_test.shape (12613,)
Repeat #0, fold #2, samples 0: X_train.shape: (113518, 33), y_train.shape (113518,), X_test.shape (12613, 33), y_test.shape (12613,)
Repeat #0, fold #3, samples 0: X_train.shape: (113518, 33), y_train.shape (113518,), X_test.shape (12613, 33), y_test.shape (12613,)
Repeat #0, fold #4, samples 0: X_train.shape: (113518, 33), y_train.shape (113518,), X_test.shape (12613, 33), y_test.shape (12613,)
Repeat #0, fold #5, samples 0: X_train.shape: (113518, 33), y_train.shape (113518,), X_test.shape (12613, 33), y_test.shape (12613,)
Repeat #0, fold #6, samples 0: X_train.shape: (113518, 33), y_train.shape (113518,), X_test.shape (12613, 33), y_test.shape (12613,)
Repeat #0, fold #7, samples 0: X_train.shape: (113518, 33), y_train.s

In [50]:
task_id = 211687
task = openml.tasks.get_task(task_id)
n_repeats, n_folds, n_samples = task.get_split_dimensions()
print(
    'Task {}: number of repeats: {}, number of folds: {}, number of samples {}.'.format(
        task_id, n_repeats, n_folds, n_samples,
    )
)


Task 211687: number of repeats: 1, number of folds: 10, number of samples 1.


In [51]:
for repeat_idx in range(n_repeats):
    for fold_idx in range(n_folds):
        for sample_idx in range(n_samples):
            train_indices, test_indices = task.get_train_test_split_indices(
                repeat=repeat_idx,
                fold=fold_idx,
                sample=sample_idx,
            )
            X_train = X.loc[train_indices]
            y_train = y[train_indices]
            X_test = X.loc[test_indices]
            y_test = y[test_indices]

            print(
                'Repeat #{}, fold #{}, samples {}: X_train.shape: {}, '
                'y_train.shape {}, X_test.shape {}, y_test.shape {}'.format(
                    repeat_idx, fold_idx, sample_idx, X_train.shape, y_train.shape, X_test.shape,
                    y_test.shape,
                )
            )

Repeat #0, fold #0, samples 0: X_train.shape: (113517, 33), y_train.shape (113517,), X_test.shape (12614, 33), y_test.shape (12614,)
Repeat #0, fold #1, samples 0: X_train.shape: (113518, 33), y_train.shape (113518,), X_test.shape (12613, 33), y_test.shape (12613,)
Repeat #0, fold #2, samples 0: X_train.shape: (113518, 33), y_train.shape (113518,), X_test.shape (12613, 33), y_test.shape (12613,)
Repeat #0, fold #3, samples 0: X_train.shape: (113518, 33), y_train.shape (113518,), X_test.shape (12613, 33), y_test.shape (12613,)
Repeat #0, fold #4, samples 0: X_train.shape: (113518, 33), y_train.shape (113518,), X_test.shape (12613, 33), y_test.shape (12613,)
Repeat #0, fold #5, samples 0: X_train.shape: (113518, 33), y_train.shape (113518,), X_test.shape (12613, 33), y_test.shape (12613,)
Repeat #0, fold #6, samples 0: X_train.shape: (113518, 33), y_train.shape (113518,), X_test.shape (12613, 33), y_test.shape (12613,)
Repeat #0, fold #7, samples 0: X_train.shape: (113518, 33), y_train.s

In [60]:
#### openml.config.start_using_configuration_for_example()

# NOTE: We are using dataset 20 from the test server: https://test.openml.org/d/20
dataset = openml.datasets.get_dataset(42252)
X, y, categorical_indicator, attribute_names = dataset.get_data(
    dataset_format='dataframe',
    target=dataset.default_target_attribute
)
clf = neighbors.KNeighborsClassifier(n_neighbors=3)
clf.fit(X, y)


ValueError: could not convert string to float: 'OMB'